In [36]:
import pandas as pd
from algos.vol_trend.bot import get_position_balance
import redis

In [68]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [69]:
replace_move

'NONE'

In [70]:
replace_perp

'SHORT'

In [71]:
details_df, balances = get_position_balance()

replace_perp = ""
replace_move = ""

try:
    if int(r.get('override_perp').decode()) == 1:
        perp = int(r.get('perp_override_direction').decode())

    if perp == 1:
        replace_perp = 'LONG'
    elif perp == -1:
        replace_perp = 'SHORT'
    else:
        replace_perp = 'NONE'
except:
    pass

try:
    if int(r.get('override_move').decode()) == 1:
        move = int(r.get('move_override_direction').decode())

    if move == 1:
        replace_move = 'LONG'
    elif move == -1:
        replace_move = 'SHORT'
    else:
        replace_move = 'NONE'
except:
    pass

new_details = pd.DataFrame()
for idx, row in details_df.iterrows():
    if row['name'] == 'BTC-PERP':
        if replace_perp != "":
            row['backtest_position'] = replace_perp
    else:
        if replace_move != "":
            if row['backtest_position'] != "NONE":
                row['backtest_position'] = replace_move
                
    new_details = new_details.append(row, ignore_index=True)

details_df = new_details

In [72]:
details_df

,backtest_date,backtest_position,entry,entry_price,name,pos_size,position
0,,NONE,0.0,NaN,BTC-MOVE-2020Q4,0.0,NONE
1,2020-12-18,NONE,0.0,6546.65,BTC-MOVE-2021Q1,0.0,NONE
2,,NONE,0.0,NaN,BTC-MOVE-2021Q2,0.0,NONE
3,2020-12-18,SHORT,0.0,22810.78,BTC-PERP,0.0,NONE


In [55]:
details_df['target_pos'] = details_df['backtest_position'].replace("LONG", 1).replace("SHORT", -1).replace("NONE", 0)

In [56]:
details_df['curr_pos'] = details_df['position'].replace("LONG", 1).replace("SHORT", -1).replace("NONE", 0)

In [57]:
to_close = details_df[details_df['target_pos'] == 0]

In [58]:
for idx, row in to_close.iterrows():
    if row['name'] == 'BTC-PERP':
        asset_type = "PERP"
    else:
        asset_type = "MOVE"
        
    if row['curr_pos'] != 0:
        print('close pos')

In [59]:
to_open = details_df[details_df['target_pos'] != 0]

In [60]:
for idx, row in to_open.iterrows():

        
    if row['target_pos'] == row['curr_pos']:
        pass
    elif row['target_pos'] * row['curr_pos'] == -1:
        print('close pos')
        print('open pos')
    else:
        print('open pos')

open pos


In [61]:
row

backtest_date        2020-12-18
backtest_position         SHORT
entry                         0
entry_price             22810.8
name                   BTC-PERP
pos_size                      0
position                   NONE
target_pos                   -1
curr_pos                      0
Name: 3, dtype: object